In [ ]:
pip install PyMuPDF gspread google-auth

In [ ]:
import fitz
import gspread
from google.oauth2.service_account import Credentials
import os
import time

# Load credentials and create a client to interact with Google Sheets
scope = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file('Path of the .json file from Google Cloud', scopes=scope)
client = gspread.authorize(creds)

# Open the Google Sheet using the sheet name
sheet = client.open("Name of the Google Sheet").sheet1

# Directory where PDF files are stored
pdf_directory = 'Path of the folder containnig PDFs'

# Function to list all PDF files in a directory
def list_pdf_files(directory):
    return [file for file in os.listdir(directory) if file.endswith('.pdf')]

# Function to extract text from PDF and handle pagination
def extract_text_with_fitz(file_path):
    doc = fitz.open(file_path)
    full_text = ''
    for page in doc:
        full_text += page.get_text()
    doc.close()
    return full_text

# Get the list of PDF files
pdf_files = list_pdf_files(pdf_directory)

# Prepare the data for batch updates
data = []
for file_name in pdf_files:
    file_path = f"{pdf_directory}/{file_name}"
    text = extract_text_with_fitz(file_path)

    # Segment text into chunks of 50K characters beacuse Google Sheet is limited to 50K
    for i in range(0, len(text), 50000):
        text_segment = text[i:i+50000]
        data.append([file_name, text_segment])

# Batch update the sheet
batch_size = 10  # Define how many rows to write at a time
time_delay = 2  # Delay in seconds to prevent rate limit issues

for start in range(0, len(data), batch_size):
    end = start + batch_size
    chunk = data[start:end]
    # Calculate the current end row based on how many items are in the chunk
    end_row = start + len(chunk) + 1  # +1 because sheet rows start at 1, not 0
    # Create the correct A1 notation range for this batch
    range = f'A{start+2}:B{end_row+1}'  # +2 to account for header row and 0-index offset
    sheet.update(range, chunk)
    time.sleep(time_delay)  # Pause to avoid hitting API limits

print("Text extraction and sheet update complete!")
